In [1]:
using JuMP
using ECOS
using GLPK
using Cbc
using MAT
using SparseArrays
using DelimitedFiles
using LinearAlgebra
using MathOptInterface

In [2]:
# getting inputs
vars = matread("R4T1.mat")
S = vars["S"]
U = vars["U"]
L = vars["L"]

4456×50 SparseMatrixCSC{Float64, Int64} with 24806 stored entries:
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿
⣿

In [3]:
size_of_v_r = 4456
size_of_v_c = 50
size_of_s_r = 2172
epsilon = 2e-5
# calculate possible zero rows of V
possible_zero_v_rows = ones(size_of_v_r)
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if U[i, j] < -epsilon || L[i, j] > epsilon
            possible_zero_v_rows[i] = 0
            break
        end
    end
end
print(sum(1-possible_zero_v_rows[i] for i in 1:size_of_v_r))
println(" number of rows of V cannot be 0")

181.0 number of rows of V cannot be 0


In [4]:
# define model linear
V = zeros(size_of_v_r, size_of_v_c)
for col in 1:size_of_v_c
    println(string("solving for col ", string(col)))
    
    model = JuMP.Model(GLPK.Optimizer)
    
    # define the problem
    @variable(model, v[1:size_of_v_r])  # define variable v
    @variable(model, v_abs[1:size_of_v_r])
    @constraint(model, v_abs .>= v)
    @constraint(model, v_abs .>= -v)
    @constraint(model, Array(L[:, col]) .<= v .<= Array(U[:, col]))  # l1 <= v <= u1

    for i in 1:size_of_s_r
        xs, vals = SparseArrays.findnz(S[i, :])
        @constraint(model, 0 == sum(vals[k] * v[xs[k]] for k in 1:length(vals)))
    end
    @objective(model, Min, sum(possible_zero_v_rows[i] * v_abs[i] for i in 1:size_of_v_r))  # min ||v||_1
    
    optimize!(model)
    println(termination_status(model))
    println(primal_status(model))
    starting_v = JuMP.value.(v)
    println("l1 problem solved. going to iterate.")
    delta = 0.002
    n = 2
    num_iteration = 5
    for iteration in 1:num_iteration
        print("doing iteration ")
        println(iteration)
        
        w = zeros(size_of_v_r)
        for i in 1:size_of_v_r
            w[i] = delta ^ n / (delta ^ n + abs(starting_v[i])^n)
        end
        for i in 1:size_of_v_r
            if abs(starting_v[i]) < epsilon && possible_zero_v_rows[i] == 1
                @constraint(model, v[i] <= starting_v[i])
            end
        end
        @objective(model, Min, sum(possible_zero_v_rows[i] * v[i] * w[i] for i in 1:size_of_v_r))
        optimize!(model)
        println(termination_status(model))
        println(primal_status(model))
        starting_v = JuMP.value.(v)
        
        # decrement n and increment delta
        n -= 1 / num_iteration
        delta += 0.3
    end
    
    for i in 1:size_of_v_r
        V[i, col] = starting_v[i]
    end
    
end

solving for col 1
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for col 2
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for col 3
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIBLE_POINT
doing iteration 3
OPTIMAL
FEASIBLE_POINT
doing iteration 4
OPTIMAL
FEASIBLE_POINT
doing iteration 5
OPTIMAL
FEASIBLE_POINT
solving for col 4
OPTIMAL
FEASIBLE_POINT
l1 problem solved. going to iterate.
doing iteration 1
OPTIMAL
FEASIBLE_POINT
doing iteration 2
OPTIMAL
FEASIB

In [5]:
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if abs(V[i, j]) < epsilon
            V[i, j] = 0
        end
    end
end

In [ ]:
# find zero rows of the weakly built V
zero_rows_of_v = ones(size_of_v_r)
for i in 1:size_of_v_r
    for j in 1:size_of_v_c
        if abs(V[i, j]) > epsilon
            zero_rows_of_v[i] = 0
            break
        end
    end
end
num_zero_v_rows = Int(sum(zero_rows_of_v[i] for i in 1:size_of_v_r))

In [ ]:
model = JuMP.Model(Cbc.Optimizer)
remaining_v_rows = Int(size_of_v_r - num_impossible_zero_v_rows - num_zero_v_rows)
@variable(model, v[1:Int(size_of_v_r - num_zero_v_rows), 1:size_of_v_c])  # define variable v
@variable(model, v_norm[1:remaining_v_rows])

println("start for")

v_norm_id = 1
v_id = 1
interesting_v_rows_ids = zeros(Int, Int(size_of_v_r - num_zero_v_rows))
for i in 1:size_of_v_r
    if zero_rows_of_v[i] == 0 && possible_zero_v_rows[i] == 1
        for j in 1:size_of_v_c
            mx_val = max(abs(L[i, j]), abs(U[i, j]))
            @constraint(model, mx_val * v_norm[v_norm_id] >= v[v_id,j])
            @constraint(model, mx_val * v_norm[v_norm_id] >= -v[v_id,j])
        end
        @constraint(model, v_norm[v_norm_id] in MathOptInterface.ZeroOne())
        v_norm_id = Int(v_norm_id+1)
    end
    if zero_rows_of_v[i] == 0
        interesting_v_rows_ids[Int(v_id)] = Int(i)
        v_id = Int(v_id + 1)
    end
end

println("endfor")

@constraint(model, Matrix(L[interesting_v_rows_ids, :]) .<= v .<= Matrix(U[interesting_v_rows_ids, :]))  # l1 <= v <= u1
modified_S = S[:, interesting_v_rows_ids]
for j in 1:size_of_v_c
    for i in 1:size_of_s_r
        xs, vals = SparseArrays.findnz(modified_S[i, :])
        if length(vals) > 0
            @constraint(model, 0 == sum(-vals[k] * v[xs[k], j] for k in 1:length(vals)))
        end
    end
    println(j)
end

@objective(model, Min, sum(v_norm[i] for i in 1:remaining_v_rows));

In [ ]:
# solve the problem, see the status of it, and store the results
optimize!(model)
println(termination_status(model))
print(primal_status(model))
starting_v_norm = JuMP.value.(v_norm)
starting_v = JuMP.value.(v)
sparse(starting_v)

In [ ]:
for i in 1:Int(size_of_v_r - num_zero_v_rows)
    for j in 1:size_of_v_c
        if abs(starting_v[i, j]) < epsilon
            starting_v[i, j] = 0
        end
        V[interesting_v_rows_ids[i], j] = starting_v[i, j]
    
    end
end
DelimitedFiles.writedlm(string("output.txt"), V, ',')